In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [23]:
from autorocks.data.loader.all_models_result_aggregator import create_all_models_comparison_dataset
from autorocks.dir_struct import LocalResultDir
from autorocks.viz import viz
from notebooks.gem5.statistics_parser import all_models_parser

output_location = "/home/salabed/workspace/latex/papers/mlsys23_bograph/figs"
output_format = "svg"  # pdf
if output_format == "svg":
    output_location = f"/tmp//svg/"
save_res = False

exp_dir = LocalResultDir / f"mobogem5/spmv_ellpack/20_params/100_iter"
model_comparison_data = create_all_models_comparison_dataset(exp_dir)
objectives = ["latency", "avg_power"]
color_palette = viz.create_color_palette_df(
    model_comparison_data.sys_observations, ["#9b59b6", "#3498db", "#95a5a6", "#e74c3c", "#34495e", "#2ecc71"]
)

model_comparison_data.sys_observations['avg_power'] = model_comparison_data.sys_observations['avg_power'].fillna(model_comparison_data.sys_observations['bench_stats.avg_power'])

model_comparison_data.sys_observations['sim_seconds'] = model_comparison_data.sys_observations['sim_seconds'].fillna(model_comparison_data.sys_observations['detailed_stats.system.sim_seconds'])


model_comparison_data.sys_observations["latency"] = 1/model_comparison_data.sys_observations['sim_seconds']
df = model_comparison_data.sys_observations.copy()


In [24]:
# import sklearn
#
# df[objectives[0]] = sklearn.preprocessing.StandardScaler().fit_transform(df[objectives[0]].values.reshape(-1, 1))
# df[objectives[1]] = sklearn.preprocessing.StandardScaler().fit_transform(df[objectives[1]].values.reshape(-1, 1))

In [42]:
from autorocks.viz.viz import OptimizationType, pareto_frontier
from autorocks.global_flags import DEVICE
from autorocks.viz import viz
from botorch.utils.multi_objective import is_non_dominated
import torch
import numpy as np
import pandas as pd

fig = pareto_frontier(
    df=df,
    optimization_type=OptimizationType.MINIMIZE,
    objectives=objectives,
    model_marker={"MOBO_SingleTaskGP": "P", "MOBoGraphStatic": "X",},
    model_palette_map=color_palette,
    title="Normalized results",
)

# fig.axes[0].set(xlim=[-1, 4], ylim=[-2, 5])
fig

In [26]:

viz.convergence_dominated_points(
    df=df,
    optimization_type=viz.OptimizationType.MINIMIZE,
    objectives=objectives,
    model_palette_map=color_palette,
    title="Number of Pareto points found. Higher is better.",
)

In [47]:
df['model'].unique()

In [48]:
from botorch.utils.multi_objective import is_non_dominated

df_mobo = df[df['model'] == 'MOBoGraphStatic']
df_boto = df[df['model'] == 'MOBO_SingleTaskGP']


y_boto = torch.tensor(df_boto[objectives].values, device="cpu") * -1
pf_boto = is_non_dominated(y_boto).cpu().numpy()
df_boto["is_non_dominated"] = False
df_boto.loc[pf_boto, "is_non_dominated"] = True



y_mobo = torch.tensor(df_mobo[objectives].values, device="cpu") * -1
pf_mobo = is_non_dominated(y_mobo).cpu().numpy()
df_mobo["is_non_dominated"] = False
df_mobo.loc[pf_mobo, "is_non_dominated"] = True



In [58]:
print(pd.concat([df_mobo[df_mobo["is_non_dominated"]][['model'] + objectives]
,
           df_boto[df_boto["is_non_dominated"]][['model'] + objectives]]).to_latex(index=False, float_format="{:0.2f}".format))

In [9]:
# Best found objective medium for each objective
figs = []
for obj in objectives:
    figs.append(
        viz.convergence_lineplot(
            df=df,  # model_comparison_data_c.system_performance,
            optimization_type=viz.OptimizationType.MINIMIZE,
            ylabel=f"{obj}",
            column_name=obj,
            model_palette_map=color_palette,
            # xlim=(10, 100),
        )
    )

In [10]:
# figs[0].axes[0].set(ylim=[0, 12.5])
figs[0]

In [11]:
# figs[1].axes[0].set(ylim=[0, 5])
figs[1]

In [18]:
for c in df.columns:
    if "sim_seconds" in c:
        print(c)

In [40]:
df.groupby(['model', 'iteration']).median()["sim_seconds"]

In [35]:
mobo_res = df.groupby(['model']).median()[objectives]

In [37]:
print(mobo_res.to_latex())

In [38]:
viz.mobo_exploration_plot(df=df, objectives=objectives, num_of_cols = 2)